Group nr: 8

Name 1 and CID: Jonatan Ornung (ornung)

Name 2 and CID: Gustav le Dous (dous)

In [1]:
import numpy as np
import pygame
from IPython.display import Image
from BalanceRobot import BalanceRobot

pygame 2.1.2 (SDL 2.0.16, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Balance robot

In this assignment, we will perform parameter estimation on a balancing robot. First, we will show how the equations of motion can be derived, then we will run a control algorithm to balance the robot. While the robot is balancing we will collect data and using this data in a regression problem we will identify some physical properties of the robot. Below we can see a drawing of the robot.


<img src="img/BallanceRobot.png" width="300"/>

| Parameter | Description | To be estimated |
| --- | --- | --- |
| $m_1$ | Mass of main body | Yes |
| $l$ | Distance from wheel hub to $m_1$ | Yes |
| $I_1$ | Moment of inertia for main body | Yes |
| $m_2$ | Mass of wheels | No |
| $r$ | Radious of wheels | No |
| $I_2$ | Moment of inertia for of wheels | No |
| - | - | - |
| $\alpha_1$ | Angle of main body | |
| $\alpha_2$ | Angle of wheel from position 0 | |


## Equations of motion

Here we show one way to calculate the equations of motion for the system. 

We start with setting up the kinematic relationships. We define $(x_1, y_1)$ to be the position for the centre of mass of the main body and $x_2$ the position of the wheels, both with respect to the coordinate frame. The kinematics equations for the system then become:

$x_2 = - \alpha_2 r$

$x_1 = - \alpha_2 r - l sin(\alpha_1)$

$y_1 = r + l cos(\alpha_1)$

$\dot{x}_1 = - r \dot{\alpha}_2 - l cos(\alpha_1) \dot{\alpha_1}$

$\dot{y}_1 = -l sin(\alpha_1) \dot{\alpha}_1$

$\dot{x}_2 = - r \dot{\alpha}_2 $ 

Potential Energy for the sytstem:

$V = l cos(\alpha_1)m_1g$

Kinematic energy:

$T = \frac{1}{2} m_1 (\dot{x}^2_1 + \dot{y}^2_1) + \frac{1}{2} m_2 \dot{x}^2_2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2$

From these the equations of motion for the balance robot can be calculated, here we use the Euler Lagrange method. First the Lagrangian is calculated by: 

$L = T - V$

and becomes:

$L = \frac{1}{2} m_1 (r^2 \dot{\alpha}^2_2 + 2 r \dot{\alpha}_2 l cos(\alpha_1) \dot{\alpha_1} + l^2  \dot{\alpha}_1^2) +
\frac{1}{2} m_2 r^2 \dot{\alpha}_2^2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2 -l cos(\alpha_1)m_1g$

Euler Lagrange equation is defined by: 

$\frac{d}{dt}(\frac{\partial L }{\partial \dot{q}_i}) - \frac{\partial L }{\partial q_i} = Q_i$

Where $q_1 = \alpha_1$, $q_2 = \alpha_2$ and $Q_i$ reprsents external force and will be torque $M$ generated by the motors. 

The equations of motion after simplification becomes:

$M = m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + m_1 l^2 \ddot{\alpha}_1 + I_1 \ddot{\alpha}_1  - lsin(\alpha_1)m_1g$

and

$-M =  m_1 r^2 \ddot{\alpha}_2 + m_1 rlcos(\alpha_1) \ddot{\alpha}_1 - m_1 rlsin(\alpha_1)\dot{\alpha}_1^2 + m_2 r^2 \ddot{\alpha}_2 + I_2 \ddot{\alpha}_2$

## Exersise 1:

To perform parameter identification with linear regression the equations need to be reformulated to the form $ y = X^T \theta $ where the vector $\theta$ will be estimated using regression. There are 3 parameters we want to estimate, $m_1$, $l$ and $I_2$, all other parameters can be assumed to be known. Rewrite the two equations of motion to the form $ y = X^T \theta $ so that $m_1$, $l$ and $I_2$ can be extracted from $\theta$. 

Hint: It might no be possible to linearize equations so that $\theta$ directly has the paramters $m_1$, $l$, $I_2$ but rather it might contain combinations of the parameters from which we later can calculate the values of $m_1$, $l$, $I_2$. 


Anwser:

$\left[ \begin{array}{c}
 -M-m_2 r^2 \ddot{\alpha}_2 - I_2 \ddot{\alpha}_2\\
 M
\end{array} \right] = 
\left[ \begin{array}{ccc}
 r^2 \ddot{\alpha}_2 & r cos(\alpha_1) \ddot{\alpha}_1 - r sin(\alpha_1) \dot{\alpha}^2_1 & 0 \\
 0  & r \ddot{\alpha}_2 cos(\alpha_1) - g sin(\alpha_1) & \ddot{\alpha}_1  \end{array} \right]
\left[ \begin{array}{c}
m_1 \\
m_1 l \\
m_1 l^2 + I_1
\end{array} \right]$



## Simulator

For those interested the equations of motion are not only interesing for the parameter identification but can also be used to build a simulator for the system. The equations of motion are rewritten to something that is easier to integrate.

$ \ddot{\alpha}_2   = \frac{-M -m_1 rlcos(\alpha_1) (M + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1) + m_1 rlsin(\alpha_1)\dot{\alpha}_1^2}{(m_1 r^2  - m_1^2 r^2 l^2 cos^2(\alpha_1)/ (m_1 l^2 + I_1) + m_2 r^2 + I_2)} $

$\ddot{\alpha}_1  = (M -  m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1)$

From these equations we can simulate the robot by either integrateing with a fixed time step or use a differential solver e.g. runge-kutta 45.  

## Data collection

Before we can do any regression on the problem we need to collect data. The data is collected while using a controller that follows a position reference signal. 


## Exercise 2:
For the first test we set the position reference signal to constant e.g $ \textit{pos_ref} = 0$. But there is one problem with using a constant reference signal. Discuss below why this should give a poor estimation of the parameters. 

Anwser: All data will look mostly the same, with disregards to noise. We need data which differ to do a meaningful regression. Now we only fit the model to model a single (Static?) state, and therefore get no preditions for other states/dynamics.



In [2]:
class Controller(object):
    def __init__(self, dt, init_states):
        self.dt = dt
        # controller parameter for position controller
        self.K_pos = 3
        self.D_pos = 0.5
        # controller parameter for angle controller
        self.K_ang = 10
        self.I_ang = 1
        self.D_ang = 1
        # limit the maximum allowed reference angle
        self.max_angle = 0.8
        # variables for the integrator in the PID controller
        self.angle_int = 0
        self.pre_o1 = init_states['a1']
    
    def update(self, t, states): 
        # The update function will be called on every timestep and return the torque for the motors. 
        # The position reference to follow
        #pos_ref =  0
        
        pos_ref = 0.1 * np.sign(np.sin(t))
        # PD controller for position
        err = self.K_pos*(pos_ref - states['pos']) - self.D_pos * states['vel']
        # The error is used as referece angle, with upper and lower bounds. 
        ref_ang = -np.clip(err, -self.max_angle, self.max_angle)
        self.angle_int += self.dt * (self.pre_o1 - states['a1'])
        self.pre_o1 = states['a1']
        # PID controller for the angle, generates the desiered motor torque. 
        M = self.K_ang*(ref_ang-states['a1']) + self.I_ang*self.angle_int - self.D_ang*states['da1']
    
        return M


## Exercise 2(.5):

In the code below we have the main loop. Your task is to save the data needed in places marked with TODO. 

In [3]:
# number of steps and step size to run the simulation/data collection
n_steps = 1000
step_time = 1e-2

# Initiate the robot
robot = BalanceRobot(dt=step_time)
# True parametes in the robot.
g = robot.p["g"] # gravity.
r = robot.p["r"] # radious of wheel.
m2 = robot.p["m2"] # mass of wheel.
I2 = robot.p["I2"] # moment of intertia of wheel.

# These parameters should only be used as ground truth
I1 = robot.p["I1"] # moment of intertia of main body.
m1 = robot.p["m1"] # mass of main body.
l =  robot.p["l"] # lenth from wheel hub to center of mass of main body. 

# Initate the controller 
state = robot.get_state()
controller = Controller(dt=step_time, init_states=state)

## TODO: Initilize X^T matrix and Y vector to hold data
#x_t = np.array([[ 0.,  0., 0.],
       
#       [ 0.,  0., 0.]])
#y= np.array([0,0])

XX_T = np.zeros((3,3))
Xy = np.zeros((3,1))

X_saved = []
Y_saved = []

## scale noise
scale_noise = 10


## Main loop 

for i in range(n_steps):
    # calculate time
    t = i*step_time
    # calculate torque 
    state = robot.get_state()
    M = controller.update(t, state)
    # step one time step for robot simulation
    dda1, dda2 = robot.step(torque=M)
    
    ## Add noice to data
    da1 = state['da1'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 1
    a1 = state['a1'] + np.random.normal(0.0, scale_noise*0.03) # alpha 1
    da2 = state['da2'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 2
    a2 = state['a2'] + np.random.normal(0.0, scale_noise*0.03) # alpha 2
    dda1 += np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 1
    dda2 += np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 2
    M += np.random.normal(0.0, scale_noise*0.05) # torque 
    
    ## TODO: save the data needed for the regression in exercise 1. 
    
    X_T = np.array([
        [r*r*dda2,r*np.cos(a1) *dda1- r*np.sin(a1)* da1* da1,0.],
        [0.,r*dda2*np.cos(a1)-g*np.sin(a1),dda1]
    ])
    Y = np.array([[-M-m2*r*r*dda2 -I2*dda2], [M]])
    
    X = np.transpose(X_T)
    
    X_saved.append(X)
    Y_saved.append(Y)
    
    XX_T += np.matmul(X, X_T)
    
    
    Xy += np.matmul(X,Y)
    
    #XX_T += np.array([[ddx_k**2, ddx_k*dx_k, ddx_k*x_k],
    #                  [ddx_k*dx_k, dx_k**2, dx_k*x_k],
    #                  [ddx_k*x_k, dx_k*x_k, x_k**2]])
    
    #Xy += np.array([[ddx_k],
    #                [dx_k],
    #                [x_k]])*y_k 
    
    
    #x_t[0,0] = r*r*dda2
    #x_t[0,1] = r*np.cos(a1) *dda1- r*np.sin(a1)* da1* da1
    #x_t[0,2] = 0
    #x_t[1,0] = 0
    #x_t[1,1] = r*dda2*np.cos(a1)-g*np.sin(a1)
    #x_t[1,2] = dda1
    
    #y[0] = -M-m2*r*2*dda2 -I2*dda2
    #y[1] = M

    
pygame.quit()

## Exercise 3: Linear regression 

Use linear regression to estimate the parameters.  

In [4]:
#Anwser: 
sol = np.linalg.inv(XX_T).dot(Xy)
print(sol)

est_m1 = sol[0,0]
est_l = sol[1,0] / est_m1
est_I1 = sol[2,0] - est_m1 * est_l*est_l

print('Estimated mass = ', np.round(est_m1, 3), ' and true  = ', robot.p['m1'])
print('Estimated length = ', np.round(est_l, 3), ' and true value = ', robot.p['l'])
print('Estimated moment of inertia = ', np.round(est_I1, 3), ' and true value = ', robot.p['I1'])



[[0.00137739]
 [0.00130523]
 [0.00670105]]
Estimated mass =  0.001  and true  =  0.5
Estimated length =  0.948  and true value =  0.15
Estimated moment of inertia =  0.005  and true value =  0.006


## Exercise 4: Calculate the cost function 

When solvning the linear regression problem we minimize the cost function. Calculate the value of the cost function on the saved data with the parameters estimated.

In [5]:
#Anwser: 


cost_value = 0
print(len(X_saved))
print(X_saved[0].shape)
print(sol.shape)
for x,y_real in zip(X_saved,Y_saved):
    
    #print(x.shape)
    #print(sol.shape)
    
    y_predicted = np.matmul(np.transpose(x), sol)
    
    cost_value += (y_real[0]-y_predicted[0]) **2 + (y_real[1] - y_predicted[1])**2

cost_value = cost_value/n_steps    

print('cost_value', cost_value)

1000
(3, 2)
(3, 1)
cost_value [0.53195987]


## Exercise 5: Lets improve the identification  

Try to change the position reference signal in the controller, try a few different ones and report the identified parameters and loss value below together with the function used. Only the first line in the controller class in the update function need to be changed. A example could be: pos_ref = 0.2*np.sin(t). 


Anwser:

| ref signal | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0 | 7,6 / 1000| -1.15| 0.009 | -0.006 | 
| 0.2*sin(t) | 7,8 / 1000 | 0.205  | 0.04 | 0.017 |  
| 0.1*sign(sin(t))  | 6.27 / 1000  |0.548 | 0.15 | 0.006 | 
|  |  | |  |  | 

## Exercise 6: Analyse cost function

Why does the cost function value not improve drastically with the better identified parameters?

Anwser: The cost function doesn't care if the paramters are good or not. Only if the regression got something which matched the data. Since the robot wasn't moving and was mostly static we probably got a really good regression. BEacause it wasn't much data to fit. Therefore the cost function is low to begin with. 


 ## Exercise 7:  Noise

Explore how the noise levels added before the data collection affects the identification results and also the value of the cost function. Why is it that the cost function reacts differenlty compared to exersice 5? Use the best performing position reference signal. 

Anwser: we used 0.1 * sign(sin(t)). With additional noise the cost function greatly increases. Because due to the noise almost no datapoints lies exactly at the avarage and are a distance away. Causing the cost function error to increase with each datapoint. 



| scale_noise | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0.1 | 0.05242534 /1000| 0.501 | 0.15 | 0.006| 
| 0.5 |1.32911155 /1000 | 0.495 | 0.15 |0.006 | 
| 1 |5.2204248 /1000| 0.447 | 0.151 |0.006 | 
| 2 |21.08854793/1000| 0.348 | 0.153 | 0.006| 
| 10 |494.19450337/1000 | 0.033 | 0.205 |0.006 | 


## Exersice 8: 

Using the intuition built from the above exercises, discuss what would be important if you where to use this method on a real world example. 

The regression need a lot of varied data to be good. The cost function does not in it self tell us how correct the parameters are. Noise in position measurements makes the model think it is really light.